In [1]:
import numpy as np
import scipy 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dataprep
import sklearn
import cartopy
import sklearn
import xgboost as xgb
import lightgbm as lgb
import catboost
import optuna
import shap
from sklearn.model_selection import RepeatedKFold, cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer

KeyboardInterrupt: 

In [ ]:
# Methods
from sklearn.linear_model import LinearRegression

# Model selection and evaluation tools
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tutorial9 import plot_feature_importance

In [ ]:
# This is to clear the warnings from the notebook, usually we should leave this on
import warnings
warnings.filterwarnings('ignore') 

# Plot settings
sns.set_context('notebook') # optimises figures for notebook display
sns.set_style('ticks') # set default plot style
colours = ['#4E79A7','#F28E2C','#E15759','#76B7B2','#59A14F', 
          '#EDC949','#AF7AA1','#FF9DA7','#9C755F','#BAB0AB']
sns.set_palette(colours) # set custom color scheme

%matplotlib inline
plt.rcParams['figure.figsize'] = (9, 6)
plt.rcParams['figure.dpi'] = 80
%config InlineBackend.figure_format = 'retina'

In [ ]:
data = pd.read_csv('listings.csv')
data.describe()

In [ ]:
data['host_acceptance_rate']

In [ ]:
data.info()
data.shape

In [ ]:
data = data[data['price'].notna()]

In [ ]:
data['price'] = data['price'].replace('[\$,]', '', regex=True).astype(float)

In [ ]:
from sklearn.model_selection import train_test_split

data['LogPrice'] = np.log(data['price'])

index_train, index_test = train_test_split(data.index, train_size=0.7, random_state=21)

train = data.loc[index_train, :].copy()
test = data.loc[index_test, :].copy()

In [ ]:
Null_variable = ['description','neighbourhood_group_cleansed','bathrooms','calendar_updated']

# EDA

In [ ]:
Useless_variable = ['id','listing_url','scrape_id','last_scraped','picture_url','host_id','host_url','host_name','host_thumbnail_url','host_picture_url','calendar_last_scraped','host_verifications','license']

In [ ]:
from dabl import detect_types

detected = detect_types(train)
detected

In [ ]:
useless_features = detected[detected['useless'] == True].index.to_list()
train[useless_features]

#host_has_profile_pic

In [ ]:
data['host_has_profile_pic'].value_counts()
sns.boxplot(x = data['host_has_profile_pic'], y = data['LogPrice'], boxprops=dict(alpha=.9), palette='Blues')
sns.despine()
plt.show()

bedroom

In [ ]:
data.loc[data['bedrooms'].notna() == True,['bedrooms','room_type','beds','bathrooms_text']]

In [ ]:
data['room_type'].value_counts()

amenities

In [ ]:
data['amenities'].value_counts()

calculated_host_listings_count_shared_rooms

In [ ]:
data['calculated_host_listings_count_shared_rooms'].value_counts()

In [ ]:
counts =data['calculated_host_listings_count_shared_rooms'].value_counts()
for level in counts.index:
    if counts[level] < 100:
        train.loc[data['calculated_host_listings_count_shared_rooms'] == level, 'calculated_host_listings_count_shared_rooms'] = '2'
        test.loc[data['calculated_host_listings_count_shared_rooms'] == level, 'calculated_host_listings_count_shared_rooms'] = '2'

In [ ]:
train['calculated_host_listings_count_shared_rooms'].value_counts()

In [ ]:
train['calculated_host_listings_count_shared_rooms'] = train['calculated_host_listings_count_shared_rooms'].astype(float)
test['calculated_host_listings_count_shared_rooms'] = test['calculated_host_listings_count_shared_rooms'].astype(float)

In [ ]:
sns.boxplot(x = train['calculated_host_listings_count_shared_rooms'], y = data['LogPrice'], boxprops=dict(alpha=.9), palette='Blues')
sns.despine()
plt.show()

In [ ]:
detected_useless_variable = ['bedrooms','amenities','host_has_profile_pic']
All_deleted_variales = detected_useless_variable + Useless_variable + Null_variable
train = train.drop(All_deleted_variales,axis = 1)
test = test.drop(All_deleted_variales,axis = 1)
data = data.drop(All_deleted_variales,axis = 1)

In [ ]:
data.info()

In [ ]:
Nominal_Variable = ['source','host_location','host_neighbourhood','property_type',
                    'room_type','neighbourhood_cleansed','neighbourhood',
                    'host_is_superhost','host_identity_verified','bathrooms_text',
                   'has_availability','instant_bookable']
Ordinal_Variable = ['host_response_time','review_scores_rating','review_scores_accuracy','review_scores_cleanliness',
                   'review_scores_checkin','review_scores_communication','review_scores_location',
                   'review_scores_value','reviews_per_month']
Discrete_Variable = ['host_listings_count','host_total_listings_count','accommodates','beds','minimum_nights',
                    'maximum_nights','minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights',
                    'maximum_maximum_nights','availability_30','availability_60','availability_90','availability_365',
                    'number_of_reviews','number_of_reviews_ltm','number_of_reviews_l30d','calculated_host_listings_count',
                    'calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms',
                    'calculated_host_listings_count_shared_rooms']
Continuous_Variable = ['host_response_rate','host_acceptance_rate','latitude','longitude','minimum_nights_avg_ntm',
                       'maximum_nights_avg_ntm']
Date_Variable = ['host_since','first_review','last_review']
Text_Variable = ['name','neighborhood_overview','host_about']
Response = ['LogPrice']

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature


median_log_price = train['LogPrice'].median()

colors = ['red' if price > median_log_price else 'blue' for price in train['LogPrice']]

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())

ax.scatter(train['longitude'], train['latitude'], c=colors, 
           edgecolors='none', alpha=0.3, transform=ccrs.PlateCarree())

ax.add_feature(cfeature.STATES.with_scale('50m'))
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))

ax.set_extent([150, 155, -35, -27], crs=ccrs.PlateCarree())

ax.set_title('House Price in NSW (training data)', fontsize=17, y=1.01, fontweight='bold')
ax.text(150.0, -33.86, 'Sydney', fontsize=8,transform=ccrs.Geodetic(), color='tab:red')
plt.show()
# Use median of log price to distinguish. Those above the median are in red, and those below the medinian are in blue.

# 1.Nominal Variable

In [ ]:
data['source'].isna().sum()

In [ ]:
sns.boxplot(x = train['source'], y = train['LogPrice'], boxprops=dict(alpha=.9), palette='Blues')
sns.despine()
plt.show()

### Therefore, we assume the previous scrape means the house has not been rented within 65 days and it has been already scraped before. And city scrape means the house for rent is newly listing

### Then we choose dummy encoding to transfer two types of scrape into 0 and 1, zero is city scrape and 1 is previous scrape. the base type is city scrape

In [ ]:
variable = 'source'
train_source_dummies = pd.get_dummies(train[variable],  prefix = variable, 
                         drop_first=True)

test_source_dummies = pd.get_dummies(test[variable],  prefix = variable, 
                         drop_first=True)
train_source_dummies = train_source_dummies.astype(float)
test_source_dummies = test_source_dummies.astype(float)

train = train.join(train_source_dummies)
test = test.join(test_source_dummies)
train.head()

### Location

In [ ]:
train[['host_location','host_neighbourhood','neighbourhood_cleansed', 'neighbourhood']].head(20)

### Because neighbourhood_cleansed is generated through latitude and longitude, it may be more accurate, host location shows the location of the host, not the location actual tenants live. Therefore, we use neighbourhood_cleansed

In [ ]:
train['neighbourhood_cleansed'].value_counts()

In [ ]:
plt.figure(figsize = (12,10))
train['neighbourhood_cleansed'].value_counts().plot(kind='barh')
sns.despine()
plt.show()

In [ ]:
neighbourhood_counts = data['neighbourhood_cleansed'].value_counts()

neighbourhood_counts_df = pd.DataFrame(neighbourhood_counts).reset_index()
neighbourhood_counts_df.columns = ['neighbourhood', 'count']

sns.boxplot(data=neighbourhood_counts_df, y='count')
plt.ylabel('Count')
plt.xlabel('Neighbourhood')
plt.show()

In [ ]:
median_count = np.median(neighbourhood_counts)
print(median_count)

In [ ]:
counts =data['neighbourhood_cleansed'].value_counts()
for level in counts.index:
    if counts[level] < 385:
        train.loc[data['neighbourhood_cleansed'] == level, 'neighbourhood_cleansed'] = 'Other'
        test.loc[data['neighbourhood_cleansed'] == level, 'neighbourhood_cleansed'] = 'Other'

In [ ]:
plt.figure(figsize = (12,10))
train['neighbourhood_cleansed'].value_counts().plot(kind='barh')
sns.despine()
plt.show()

In [ ]:
index_train1, index_valid1 = train_test_split(train.index, train_size=0.7, random_state=21)

train1 = train.loc[index_train1, :].copy()
valid1 = train.loc[index_valid1, :].copy()

y_train_1 = train1['LogPrice']
y_valid_1 = valid1['LogPrice']

In [ ]:
from category_encoders.target_encoder import TargetEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder #eliminate data leakage
from category_encoders.glmm import GLMMEncoder
from category_encoders.cat_boost import CatBoostEncoder

# This cell is a bit repetitive to keep the code simpler

predictor = ['neighbourhood_cleansed']

encoder = TargetEncoder().fit(train1[predictor], y_train_1) # we must use the training set for this
train1['NeighborhoodTE'] = encoder.transform(train1[predictor])
valid1['NeighborhoodTE'] = encoder.transform(valid1[predictor])

encoder = LeaveOneOutEncoder().fit(train1[predictor], y_train_1) 
train1['NeighborhoodLeaveOneOut'] = encoder.transform(train1[predictor])
valid1['NeighborhoodLeaveOneOut'] = encoder.transform(valid1[predictor])

encoder =  GLMMEncoder().fit(train1[predictor], y_train_1)
train1['NeighborhoodGLMM'] = encoder.transform(train1[predictor])
valid1['NeighborhoodGLMM'] = encoder.transform(valid1[predictor])

encoder = CatBoostEncoder().fit(train1[predictor], y_train_1) 
train1['NeighborhoodCatBoost'] = encoder.transform(train1[predictor])
valid1['NeighborhoodCatBoost'] = encoder.transform(valid1[predictor])

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
rows = ['Target encoder', 'Leave-one-out encoder', 
      'GLMM encoder', 'CatBoost encoder']

columns=['RMSE', 'R-Squared', 'MAE']

results = pd.DataFrame(0.0, columns=columns, index=rows)

for i in range(len(rows)):

    if i==0: 
        predictors = ['NeighborhoodTE'] 
    elif i==1:
        predictors = ['NeighborhoodLeaveOneOut']
    elif i==2: 
        predictors = ['NeighborhoodGLMM']
    elif i==3: 
        predictors = ['NeighborhoodCatBoost']  
    
    X_train_1 =  train1[predictors]
    X_valid_1 = valid1[predictors]
    
    # Fit model
    ols = LinearRegression().fit(X_train_1, y_train_1)

    # validation set predictions
    y_pred_1 = ols.predict(X_valid_1)
    results.iloc[i, 0] = np.sqrt(mean_squared_error(y_valid_1, y_pred_1))
    results.iloc[i, 1] = r2_score(y_valid_1, y_pred_1)
    results.iloc[i, 2] = mean_absolute_error(y_valid_1, y_pred_1)

results.round(3)

In [ ]:
data[['property_type','room_type']].isna().sum().sort_values(ascending = False)

In [ ]:
data[['property_type','room_type']]

In [ ]:
train['property_type'].value_counts()

In [ ]:
data['property_type'] = data['property_type'].apply(lambda x: x.split()[-1])
data['property_type'].value_counts()

In [ ]:
train['property_type'] = train['property_type'].apply(lambda x: x.split()[-1])
test['property_type'] = test['property_type'].apply(lambda x: x.split()[-1])

In [ ]:
pro_counts =data['property_type'].value_counts()
for level in pro_counts.index:
    if pro_counts[level] < 40:
        train.loc[data['property_type'] == level, 'property_type'] = 'Other'
        test.loc[data['property_type'] == level, 'property_type'] = 'Other'

In [ ]:
plt.figure(figsize = (18,10))
sns.boxplot(x = train['property_type'], y = train['LogPrice'], boxprops=dict(alpha=.9), palette='Blues')
sns.despine()
plt.show()

In [ ]:
variable = 'property_type'
train_property_dummies = pd.get_dummies(train[variable], prefix = variable)

test_property_dummies = pd.get_dummies(test[variable], prefix = variable)

train_property_dummies = train_property_dummies.astype(float)
test_property_dummies = test_property_dummies.astype(float)

train = train.join(train_property_dummies)
test = test.join(test_property_dummies)
train.head()

In [ ]:
train_property_dummies.head()

In [ ]:
test_property_dummies.head()

In [ ]:
train['room_type'].value_counts().head(30)

In [ ]:
test['room_type'].value_counts().head(30)

In [ ]:
plt.figure(figsize = (16,10))
sns.boxplot(x = train['room_type'], y = train['LogPrice'], boxprops=dict(alpha=.9), palette='Blues')
sns.despine()
plt.show()

In [ ]:
variable = 'room_type'
train_room_dummies = pd.get_dummies(train[variable],  prefix = variable)

test_room_dummies = pd.get_dummies(test[variable],  prefix = variable)

train_room_dummies = train_room_dummies.astype(float)
test_room_dummies = test_room_dummies.astype(float)

train = train.join(train_room_dummies)
test = test.join(test_room_dummies)
train.head()

In [ ]:
train['host_is_superhost'].isna()

In [ ]:
from dataprep.eda.missing import plot_missing
plot_missing(train['host_is_superhost'],display  = ['Stats', 'Bar Chart'])

In [ ]:
train['host_is_superhost'] = train['host_is_superhost'].fillna('NotEvaluate')

In [ ]:
test['host_is_superhost'] = test['host_is_superhost'].fillna('NotEvaluate')

In [ ]:
variable = 'host_is_superhost'
train_host_dummies = pd.get_dummies(train[variable],  prefix = variable, 
                         drop_first=True)

test_host_dummies = pd.get_dummies(test[variable],  prefix = variable, 
                         drop_first=True)

train_host_dummies = train_host_dummies.astype(float)
test_host_dummies = test_host_dummies.astype(float)

train = train.join(train_host_dummies)
test = test.join(test_host_dummies)

In [ ]:
train.head()

In [ ]:
from dataprep.eda.missing import plot_missing
plot_missing(train['host_identity_verified'],display  = ['Stats', 'Bar Chart'])

In [ ]:
keys = {
        'f':0 , # Failed to pass verification
        't':1, # verified
}
train['host_identity_verified'] = train['host_identity_verified'].replace(keys)
test['host_identity_verified'] = test['host_identity_verified'].replace(keys)

In [ ]:
train['host_identity_verified']

In [ ]:
train['bathrooms_text'].isna().sum()

In [ ]:
rows_bathroom_nan = train.loc[train['bathrooms_text'].isna()]

In [ ]:
rows_bathroom_nan.iloc[:,15:]

In [ ]:
train['bathrooms_text'] = train.apply(lambda row: str(row['accommodates'] / 2) + ' bathrooms' if pd.isna(row['bathrooms_text']) else row['bathrooms_text'], axis=1)

In [ ]:
train['bathrooms_text'].isna().sum()

In [ ]:
def replace_non_numeric(value):
    if "half"in value.lower():
        return 0.5 
    else:
        return float(value.split()[0]) 

train['bathrooms_text'] = train['bathrooms_text'].apply(replace_non_numeric)

In [ ]:
train['bathrooms_text']

In [ ]:
test['bathrooms_text'] = test.apply(lambda row: str(row['accommodates'] / 2) + ' bathrooms' if pd.isna(row['bathrooms_text']) else row['bathrooms_text'], axis=1)

In [ ]:
test['bathrooms_text'] = test['bathrooms_text'].apply(replace_non_numeric)

In [ ]:
train['has_availability'].isna().sum()

In [ ]:
keys = {
        'f':0 , # cannot automatically book
        't':1, # can automatically book
}
train['has_availability'] = train['has_availability'].replace(keys)
test['has_availability'] = test['has_availability'].replace(keys)

In [ ]:
train['instant_bookable'].isna().sum()

In [ ]:
keys = {
        'f':0 , # not available
        't':1, # available
}
train['instant_bookable'] = train['instant_bookable'].replace(keys)
test['instant_bookable'] = test['instant_bookable'].replace(keys)

# 2. Ordinal Variable

In [ ]:
Ordinal_Variable

## 2.1 host_response_time

In [ ]:
data[['host_response_time','host_response_rate','host_acceptance_rate']].isna()

In [ ]:
data['host_response_time'].value_counts()

In [ ]:
train['host_response_time'] = train['host_response_time'].fillna('No yet reviewed')
test['host_response_time'] = test['host_response_time'].fillna('No yet reviewed')

In [ ]:
sns.boxplot(x = train['host_response_time'], y = train['LogPrice'], boxprops=dict(alpha=.9), palette='Blues')
sns.despine()
plt.show()

In [ ]:
keys = {
        'No yet reviewed': 0,
        'within an hour': 1,
        'within a few hours': 2,
        'within a day': 3,
        'a few days or more': 4, 
}

variable = 'host_response_time'
train[variable] = train[variable].replace(keys)
test[variable] = test[variable].replace(keys)

## 2.2 reviews_per_month

In [ ]:
data['reviews_per_month'].value_counts()

In [ ]:
train['reviews_per_month'].isna().sum()

In [ ]:
def reviews(x):
    if x > 5:
        return 4
    elif x > 3:
        return 3
    elif x > 1:
        return 2
    elif x > 0:
        return 1
    else:
        return 0
train['reviews_per_month'] = train['reviews_per_month'].apply(reviews)
test['reviews_per_month'] = test['reviews_per_month'].apply(reviews)
train['reviews_per_month'].value_counts()

In [ ]:
sns.boxplot(x = train['reviews_per_month'], y = train['LogPrice'], boxprops=dict(alpha=.9), palette='Blues')
sns.despine()
plt.show()

## 2.3 Score

In [ ]:
score = ['review_scores_rating','review_scores_accuracy','review_scores_cleanliness',
                   'review_scores_checkin','review_scores_communication','review_scores_location',
                   'review_scores_value']

In [ ]:
plt.figure(figsize=(12, 9))
corr = train[score].corr()
n_var = len(corr) # Number of variables

# Plot the heatmap
plt.imshow(corr, cmap = 'viridis',vmin=-1, vmax=1)

plt.xticks(range(n_var), corr.columns,rotation=90)
plt.yticks(range(n_var), corr.columns)

for i in range(n_var):
    for j in range(n_var):
        #              format to 2 decimal places  ,  horizonta/vertical align center  
        plt.text(i, j, '{:.2f}'.format(corr.iloc[i, j]), ha="center", va="center", color="k", fontsize = 22)

plt.colorbar()

plt.title('Correlations')
plt.savefig('correlation.png')
# All variables of score-related are highly correlated. 
# Hence I choose the 'review_scores_rating' as the baseline since it is the most representative variable.

In [ ]:
train[['review_scores_rating','first_review','last_review']]

In [ ]:
def rate_standardized(x):
    if x > 4.5:
        return 5
    elif x > 3.5:
        return 4
    elif x > 2.5:
        return 3
    elif x > 1.5:
        return 2
    elif x > 0.5:
        return 1
    elif x > 0:
        return 0
    else:
        return -1 #-1 means not yet reviewed
    
for col in score:
    train[col] = train[col].apply(rate_standardized)
    test[col] = test[col].apply(rate_standardized)


In [ ]:
train['review_scores_rating'].value_counts()

In [ ]:
sns.boxplot(x = train['review_scores_rating'], y = train['LogPrice'], boxprops=dict(alpha=.9), palette='Blues')
sns.despine()
plt.show()

# 3. Discrete Variable

## 3.1 Discrete variable correlation analysis and data frequency presentation

In [ ]:
from dataprep.eda import plot_correlation

plot_correlation(train[Discrete_Variable + Response], 'LogPrice')

In [ ]:
from dataprep.eda import plot
plot(train[Discrete_Variable])

In [ ]:
plt.figure(figsize=(12, 10))  
corr = train[Discrete_Variable].corr()


plt.imshow(corr, cmap='coolwarm', vmin=-1, vmax=1)  


plt.xticks(range(len(corr)), corr.columns, rotation=90)  
plt.yticks(range(len(corr)), corr.columns)


plt.colorbar()

plt.title('Correlations')
plt.savefig('correlation.png')
plt.show()

## 3.2 Missing value

In [ ]:
train[['host_listings_count','host_total_listings_count','accommodates','beds','minimum_nights',
                    'maximum_nights','minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights',
                    'maximum_maximum_nights','availability_30','availability_60','availability_90','availability_365',
                    'number_of_reviews','number_of_reviews_ltm','number_of_reviews_l30d','calculated_host_listings_count',
                    'calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms',
                    'calculated_host_listings_count_shared_rooms']].isna().sum()

In [ ]:
from dataprep.eda.missing import plot_missing
plot_missing(train['beds'],display  = ['Stats', 'Bar Chart'])

In [ ]:
missing_beds_train = train['beds'].isna()
missing_data_train = train[missing_beds_train][['accommodates']]
train.loc[missing_beds_train, 'beds'] = missing_data_train['accommodates']

missing_beds_test = test['beds'].isna()
missing_data_test = test[missing_beds_test][['accommodates']]
test.loc[missing_beds_test, 'beds'] = missing_data_test['accommodates']

# Result
print("Number of missing values in 'beds' column in train set:", train['beds'].isna().sum())
print("Number of missing values in 'beds' column in test set:", test['beds'].isna().sum())

## 3.3 listings_count_stats and Total_Listing_number


In [ ]:
listings_count_stats = train['host_listings_count'].describe()

print(listings_count_stats)

In [ ]:
Q1_host = train['host_listings_count'].quantile(0.25)
Q3_host = train['host_listings_count'].quantile(0.75)
IQR_host = Q3_host - Q1_host

lower_limit_host = Q1_host - 1.5 * IQR_host
upper_limit_host = Q3_host + 1.5 * IQR_host

train['host_listings_count'] = train['host_listings_count'].apply(lambda x: upper_limit_host if x > upper_limit_host else (lower_limit_host if x < lower_limit_host else x))

test['host_listings_count'] = test['host_listings_count'].apply(lambda x: upper_limit_host if x > upper_limit_host else (lower_limit_host if x < lower_limit_host else x))

print(train['host_listings_count'].describe())

In [ ]:
Total_listings_count= train['host_total_listings_count'].describe()

print(Total_listings_count)

In [ ]:
Q1_total = train['host_total_listings_count'].quantile(0.25)
Q3_total = train['host_total_listings_count'].quantile(0.75)
IQR_total = Q3_total - Q1_total

lower_limit_total = Q1_total - 1.5 * IQR_total
upper_limit_total = Q3_total + 1.5 * IQR_total

train['host_total_listings_count'] = train['host_total_listings_count'].apply(lambda x: upper_limit_total if x > upper_limit_total else (lower_limit_total if x < lower_limit_total else x))

test['host_total_listings_count'] = test['host_total_listings_count'].apply(lambda x: upper_limit_total if x > upper_limit_total else (lower_limit_total if x < lower_limit_total else x))

print(train['host_total_listings_count'].describe())

## 3.4 Beds and accommodates

In [ ]:
train['beds'].describe()

In [ ]:
plt.figure(figsize=(8, 6))
plt.boxplot(train['beds'], vert=False)
plt.xlabel('beds_number')
plt.title('Boxplot of beds_number')
plt.show()

In [ ]:
Q1_beds = train['beds'].quantile(0.25)
Q3_beds = train['beds'].quantile(0.75)
IQR_beds = Q3_beds - Q1_beds

lower_limit_beds = Q1_beds - 1.5 * IQR_beds
upper_limit_beds = Q3_beds + 1.5 * IQR_beds

train['beds'] = train['beds'].apply(lambda x: upper_limit_beds if x > upper_limit_beds else (lower_limit_beds if x < lower_limit_beds else x))
test['beds'] = test['beds'].apply(lambda x: upper_limit_beds if x > upper_limit_beds else (lower_limit_beds if x < lower_limit_beds else x))

trimmed_counts = train['beds'].value_counts()
print("Number of occurrences of each value after processing:")
print(trimmed_counts)

In [ ]:
import re
test['accommodates'] = test['accommodates'].apply(lambda x: float(re.findall(r'\d+\.?\d*', str(x))[0]) if pd.notnull(x) else np.nan)

large_rooms_accommodates_test = test[test['accommodates'] > 10]

print("Data for rooms with more than 10 people in the test set:")
print(large_rooms_accommodates_test[['beds', 'accommodates']])

excess_accommodates_test = large_rooms_accommodates_test[large_rooms_accommodates_test['accommodates'] > 2 * large_rooms_accommodates_test['beds']]

excess_accommodates_test['accommodates'] = excess_accommodates_test['beds'] * 2

replacement_count_test = excess_accommodates_test.shape[0]

print("Replaced accommodates in the test set:", replacement_count_test)




## 3.5 minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(18, 10))


columns_1125 = ['minimum_nights', 'maximum_nights', 
           'minimum_minimum_nights', 'maximum_minimum_nights', 
           'minimum_maximum_nights', 'maximum_maximum_nights']

for i, column in enumerate(columns_1125):
    row = i // 3
    col = i % 3
    axs[row, col].hist(train[column], bins=30, color='skyblue', edgecolor='black')
    axs[row, col].set_title(f'Histogram of {column}')
    axs[row, col].set_xlabel(column)
    axs[row, col].set_ylabel('Frequency')
    axs[row, col].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
selected_columns = ['minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 
                    'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights']

for column in selected_columns:
    train.loc[train[column] > 1125, column] = 1125

for column in selected_columns:
    test.loc[test[column] > 1125, column] = 1125

def classify_nights(x):
    if x <= 30:
        return 0
    elif x <= 60:
        return 1
    elif x <= 90:
        return 2
    else:
        return 3

def classify_max_nights(x):
    if x >= 1125:
        return 1
    elif x >= 730:
        return 2
    elif x >= 365:
        return 3
    else:
        return 0

for column in ['minimum_minimum_nights', 'minimum_nights', 'maximum_minimum_nights']:
    train[f'{column}_class'] = train[column].apply(classify_nights)
    test[f'{column}_class'] = test[column].apply(classify_nights)

for column in ['minimum_maximum_nights', 'maximum_maximum_nights', 'maximum_nights']:
    train[f'{column}_class'] = train[column].apply(classify_max_nights)  
    test[f'{column}_class'] = test[column].apply(classify_max_nights)

for column in ['minimum_minimum_nights', 'minimum_nights', 'maximum_minimum_nights']:
    print(f"Train {column} class counts:")
    print(train[f'{column}_class'].value_counts())
    print("")

for column in ['minimum_maximum_nights', 'maximum_maximum_nights', 'maximum_nights']:
    print(f"Train {column} class counts:")
    print(train[f'{column}_class'].value_counts())
    print("")

## 3.5availability_30', 'availability_60', 'availability_90', 'availability_365

In [ ]:
selected_columns_availability = ['availability_30', 'availability_60', 'availability_90', 'availability_365']

for column in selected_columns_availability:
    counts = train[column].value_counts()
    print(f"{column} number of occurrences:")
    print(counts)

In [ ]:
selected_columns_availability = ['availability_30', 'availability_60', 'availability_90', 'availability_365']

for column in selected_columns_availability:
    train[column] = train[column].apply(lambda x: 1 if x != 0 else 0)

for column in selected_columns_availability:
    test[column] = test[column].apply(lambda x: 1 if x != 0 else 0)

for column in selected_columns_availability:
    print(f"{column} number of occurrences after processing in train set:")
    print(train[column].value_counts())
    print()

## 3.6 number_of_reviews','number_of_reviews_ltm','number_of_reviews_l30d

In [ ]:
variables = ['number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d']


plt.figure(figsize=(12, 6))
for i, variable in enumerate(variables, 1):
    plt.subplot(1, 3, i)
    plt.boxplot(train[variable], vert=False)
    plt.xlabel(variable)
    plt.title(f'Boxplot of {variable}')
    

    median = train[variable].median()
    mean = train[variable].mean()
    plt.axvline(median, color='r', linestyle='--', label=f'Median: {median:.2f}')
    plt.axvline(mean, color='g', linestyle='--', label=f'Mean: {mean:.2f}')
    plt.legend()


plt.tight_layout()
plt.show()

In [ ]:
def classify_reviews_ltm(num_reviews_ltm, median_reviews_ltm, mean_reviews_ltm):
    if num_reviews_ltm < median_reviews_ltm:
        return 1
    elif num_reviews_ltm == median_reviews_ltm:
        return 2
    elif num_reviews_ltm < mean_reviews_ltm:
        return 3
    else:
        return 4

# 处理 'number_of_reviews_ltm'
median_reviews_ltm = train['number_of_reviews_ltm'].median()
mean_reviews_ltm = train['number_of_reviews_ltm'].mean()

train['number_of_reviews_ltm'] = train['number_of_reviews_ltm'].apply(classify_reviews_ltm,
                                                                     args=(median_reviews_ltm, mean_reviews_ltm))

print("Review Category Counts for 'number_of_reviews_ltm' in the train set:")
print(train['number_of_reviews_ltm'].value_counts())

test['number_of_reviews_ltm'] = test['number_of_reviews_ltm'].apply(classify_reviews_ltm,
                                                                   args=(median_reviews_ltm, mean_reviews_ltm))

In [ ]:
def classify_reviews(num_reviews, median_reviews, mean_reviews):
    if num_reviews < median_reviews:
        return 1
    elif num_reviews == median_reviews:
        return 2
    else:
        return 3

# 处理 'number_of_reviews_ltm'
median_reviews_ltm = train['number_of_reviews_ltm'].median()
mean_reviews_ltm = train['number_of_reviews_ltm'].mean()

train['number_of_reviews_ltm'] = train['number_of_reviews_ltm'].apply(classify_reviews,
                                                                     args=(median_reviews_ltm, mean_reviews_ltm))

print("Review Category Counts for 'number_of_reviews_ltm' in the train set:")
print(train['number_of_reviews_ltm'].value_counts())

test['number_of_reviews_ltm'] = test['number_of_reviews_ltm'].apply(classify_reviews,
                                                                   args=(median_reviews_ltm, mean_reviews_ltm))


In [ ]:

plt.figure(figsize=(8, 6))

# Need to judge how to classify by images (special to the first two because the box-and-line diagrams are not )

plt.hist(train['number_of_reviews_l30d'], bins=30, color='skyblue', edgecolor='black')

plt.title('Distribution of number_of_reviews_l30d')
plt.xlabel('Number of Reviews (Last 30 Days)')
plt.ylabel('Frequency')

plt.show()


In [ ]:
def classify_reviews(x):
    if x <= 2:
        return 0
    elif x <= 4:
        return 1
    else:
        return 2

train['review_level_l30d'] = train['number_of_reviews_l30d'].apply(classify_reviews)
test['review_level_l30d'] = test['number_of_reviews_l30d'].apply(classify_reviews)
review_level_counts = train['review_level_l30d'].value_counts()

print(review_level_counts)

In [ ]:
train["review_level_l30d"].dtype

## 3.7 calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'

In [ ]:

train['calculated_host_listings_count'] = train['calculated_host_listings_count'].apply(lambda x: 0 if x == 1 else 1)


test['calculated_host_listings_count'] = test['calculated_host_listings_count'].apply(lambda x: 0 if x == 1 else 1)

In [ ]:
def classify_value(x):
    if x == 0:
        return 0
    elif x == 1:
        return 1
    else:
        return 2

def apply_classification(df):
    df['entire_homes_category'] = df['calculated_host_listings_count_entire_homes'].apply(classify_value)
    df['private_rooms_category'] = df['calculated_host_listings_count_private_rooms'].apply(classify_value)
    df['shared_rooms_category'] = df['calculated_host_listings_count_shared_rooms'].apply(classify_value)
    return df

train = apply_classification(train)

test = apply_classification(test)


In [ ]:
entire_homes_counts_train = train['entire_homes_category'].value_counts()

private_rooms_counts_train = train['private_rooms_category'].value_counts()
shared_rooms_counts_train = train['shared_rooms_category'].value_counts()
calculated_host_counts_train = train['calculated_host_listings_count'].value_counts()


plt.figure(figsize=(15, 6))

plt.subplot(1, 4, 1)
entire_homes_counts_train.plot(kind='bar', color='blue')
plt.title('Entire Homes Category')
plt.xlabel('Category')
plt.ylabel('Count')

plt.subplot(1, 4, 2)
private_rooms_counts_train.plot(kind='bar', color='green')
plt.title('Private Rooms Category')
plt.xlabel('Category')
plt.ylabel('Count')

plt.subplot(1, 4, 3)
shared_rooms_counts_train.plot(kind='bar', color='orange')
plt.title('Shared Rooms Category')
plt.xlabel('Category')
plt.ylabel('Count')

plt.subplot(1, 4, 4)
calculated_host_counts_train.plot(kind='bar', color='red')
plt.title('Calculated Host Listings Count Category')
plt.xlabel('Category')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
unique_values = train['calculated_host_listings_count_shared_rooms'].unique()
print(unique_values)

# 4 Continuous Variable

In [ ]:
Continuous_Variable

## 4.1 Some EDA

### 4.1.1 host_response_rate

In [ ]:
train['host_response_rate'] = train['host_response_rate'].str.rstrip('%').astype(float)
test['host_response_rate'] = test['host_response_rate'].str.rstrip('%').astype(float)

In [ ]:
host_response_rate_train = train['host_response_rate'].mean()
host_response_rate_test = test['host_response_rate'].mean()
train['host_response_rate'] = train['host_response_rate'].fillna(host_response_rate_train)
test['host_response_rate'] = test['host_response_rate'].fillna(host_response_rate_test)

### 4.1.2 host_acceptance_rate

In [ ]:
train['host_acceptance_rate'] = train['host_acceptance_rate'].str.rstrip('%').astype(float)
test['host_acceptance_rate'] = test['host_acceptance_rate'].str.rstrip('%').astype(float)

In [ ]:
host_acceptance_rate_train = train['host_acceptance_rate'].mean()
host_acceptance_rate_test = test['host_acceptance_rate'].mean()
train['host_acceptance_rate'] = train['host_acceptance_rate'].fillna(host_acceptance_rate_train)
test['host_acceptance_rate'] = test['host_acceptance_rate'].fillna(host_acceptance_rate_test)

### 4.1.3 maximum_nights_avg_ntm

In [ ]:
train['maximum_nights_avg_ntm'].isna().sum()

### 4.1.4 minimum_nights_avg_ntm

In [ ]:
train['minimum_nights_avg_ntm'].isna().sum()

In [ ]:
plt.figure(figsize=(12, 10)) 
corr = train[Continuous_Variable].corr()


plt.imshow(corr, cmap='coolwarm', vmin=-1, vmax=1)  

plt.xticks(range(len(corr)), corr.columns, rotation=90)  
plt.yticks(range(len(corr)), corr.columns)

plt.colorbar()

plt.title('Correlations')
plt.savefig('correlation.png')
plt.show()

In [ ]:
from dataprep.eda import plot_correlation

plot_correlation(train[Continuous_Variable + Response], 'LogPrice')

In [ ]:
sorted_train = train.sort_values(by='maximum_nights_avg_ntm', ascending=False)

In [ ]:
sorted_train['maximum_nights_avg_ntm']

In [ ]:
train.loc[train['maximum_nights_avg_ntm'] > 20000, 'maximum_nights_avg_ntm'] = 1125
test.loc[test['maximum_nights_avg_ntm'] > 20000, 'maximum_nights_avg_ntm'] = 1125

In [ ]:
from dataprep.eda import plot

plot(train[Continuous_Variable])

In [ ]:
def minimum_nights(x):
    if x >= 90:
        return 'Longer than 3 months'
    elif x >= 30:
        return 'Longer than 1 month but less than 3 months'
    else:
        return 'Less than 1 month'
train['minimum_nights_avg_ntm'] = train['minimum_nights_avg_ntm'].apply(minimum_nights)
test['minimum_nights_avg_ntm'] = test['minimum_nights_avg_ntm'].apply(minimum_nights)
train['minimum_nights_avg_ntm'].value_counts()

In [ ]:
train['minimum_nights_avg_ntm'] = train['minimum_nights_avg_ntm'].replace({'Less than 1 month': 1, 'Longer than 1 month but less than 3 months': 2, 'Longer than 3 months': 3})
test['minimum_nights_avg_ntm'] = test['minimum_nights_avg_ntm'].replace({'Less than 1 month': 1, 'Longer than 1 month but less than 3 months': 2, 'Longer than 3 months': 3})

In [ ]:
def maximum_nights(x):
    if x >= 1125:
        return 'Longer than 3 years and 1 month'
    elif x >= 730:
        return 'Longer than 2 years but less than 3 years and 1 month'
    elif x >= 365:
        return 'Longer than 1 year but less than 2 years'
    else:
        return 'Less than 1 year'
train['maximum_nights_avg_ntm'] = train['maximum_nights_avg_ntm'].apply(maximum_nights)
test['maximum_nights_avg_ntm'] = test['maximum_nights_avg_ntm'].apply(maximum_nights)
train['maximum_nights_avg_ntm'].value_counts()

In [ ]:
train['maximum_nights_avg_ntm'] = train['maximum_nights_avg_ntm'].replace({'Less than 1 year': 1, 'Longer than 1 year but less than 2 years': 2, 'Longer than 2 years but less than 3 years and 1 month': 3, 'Longer than 3 years and 1 month': 4})
test['maximum_nights_avg_ntm'] = test['maximum_nights_avg_ntm'].replace({'Less than 1 year': 1, 'Longer than 1 year but less than 2 years': 2, 'Longer than 2 years but less than 3 years and 1 month': 3, 'Longer than 3 years and 1 month': 4})

In [ ]:
sns.boxplot(x = train['minimum_nights_avg_ntm'], y = train['LogPrice'], boxprops=dict(alpha=.9), palette='Blues')
sns.despine()
plt.show()

In [ ]:
plt.figure(figsize=(18, 10))
sns.boxplot(x = train['maximum_nights_avg_ntm'], y = train['LogPrice'], boxprops=dict(alpha=.9), palette='Blues')
sns.despine()
plt.show()

## 4.2 Transformation

### 4.2.1 host_response_rate

In [ ]:
train['host_response_rate'].isna().sum()

In [ ]:
sns.regplot(x = train['host_response_rate'], y = train['LogPrice'], 
            lowess=True, line_kws={'color':'black', 'alpha':0.6})

sns.despine()
plt.show()

In [ ]:
train['Loghost_response_rate'] = np.log(train['host_response_rate'])
test['Loghost_response_rate'] = np.log(test['host_response_rate'])

index_train2, index_valid2 = train_test_split(train.index, train_size=0.7, random_state=21)

train2 = train.loc[index_train2, :].copy()
valid2 = train.loc[index_valid2, :].copy()

train2['Loghost_response_rate'].fillna(0, inplace=True)
valid2['Loghost_response_rate'].fillna(0, inplace=True)
train2['Loghost_response_rate'] = train2['Loghost_response_rate'].replace(-np.inf, 0)
valid2['Loghost_response_rate'] = valid2['Loghost_response_rate'].replace(-np.inf, 0)

y_train_2 = train2['LogPrice']
y_valid_2 = valid2['LogPrice']


sns.regplot(x = train['Loghost_response_rate'], y = train['LogPrice'], 
            lowess=True,  line_kws={'color':'black', 'alpha':0.6})

sns.despine()
plt.show()


In [ ]:
from sklearn.preprocessing import PowerTransformer

yj = PowerTransformer(method='yeo-johnson')
train2['host_response_rateYJ']  = yj.fit_transform(train2[['host_response_rate']])
valid2['host_response_rateYJ'] = yj.transform(valid2[['host_response_rate']])

bc = PowerTransformer(method='box-cox')
train2['host_response_rateBC']  = yj.fit_transform(train2[['host_response_rate']])
valid2['host_response_rateBC'] = yj.transform(valid2[['host_response_rate']])

In [ ]:
plot(train2, 'host_response_rateYJ')

In [ ]:
plot(train2, 'host_response_rateBC')

In [ ]:
# Initialise table
rows = ['No transformation', 'Log transformation', 'Yeo-Johnson transformation','Box-cox transformation']
columns=['RMSE', 'R-Squared', 'MAE']
results = pd.DataFrame(0.0, columns=columns, index=rows)

for i in range(len(rows)):

    # Design matrices
    if i==0:
        predictor = ['host_response_rate']
    elif i==1:
        predictor = ['Loghost_response_rate']
    elif i==2:
        predictor = ['host_response_rateYJ']
    else: 
        predictor = ['host_response_rateBC']
    
    X_train2 =  train2[predictor]
    X_valid2 = valid2[predictor]
    
    # Fit model
    ols = LinearRegression().fit(X_train2, y_train_2)

    # Validation set predictions
    y_pred2 = ols.predict(X_valid2)
    
    # Validation metrics
    results.iloc[i, 0] = np.sqrt(mean_squared_error(y_valid_2, y_pred2))
    results.iloc[i, 1] = r2_score(y_valid_2, y_pred2)
    results.iloc[i, 2] = mean_absolute_error(y_valid_2, y_pred2)

results.round(3)

### 4.2.2 host_acceptance_rate

In [ ]:
train['host_acceptance_rate'].isna().sum()

In [ ]:
sns.regplot(x = train['host_acceptance_rate'], y = train['LogPrice'], 
            lowess=True, line_kws={'color':'black', 'alpha':0.6})

sns.despine()
plt.show()

In [ ]:
train['Loghost_acceptance_rate'] = np.log(train['host_acceptance_rate'])
test['Loghost_acceptance_rate'] = np.log(test['host_acceptance_rate'])

index_train2, index_valid2 = train_test_split(train.index, train_size=0.7, random_state=21)

train2 = train.loc[index_train2, :].copy()
valid2 = train.loc[index_valid2, :].copy()

train2['Loghost_acceptance_rate'].fillna(0, inplace=True)
valid2['Loghost_acceptance_rate'].fillna(0, inplace=True)
train2['Loghost_acceptance_rate'] = train2['Loghost_acceptance_rate'].replace(-np.inf, 0)
valid2['Loghost_acceptance_rate'] = valid2['Loghost_acceptance_rate'].replace(-np.inf, 0)

y_train_2 = train2['LogPrice']
y_valid_2 = valid2['LogPrice']


sns.regplot(x = train['Loghost_acceptance_rate'], y = train['LogPrice'], 
            lowess=True,  line_kws={'color':'black', 'alpha':0.6})

sns.despine()
plt.show()

In [ ]:
from sklearn.preprocessing import PowerTransformer

yj = PowerTransformer(method='yeo-johnson')
train2['host_acceptance_rateYJ']  = yj.fit_transform(train2[['host_acceptance_rate']])
valid2['host_acceptance_rateYJ'] = yj.transform(valid2[['host_acceptance_rate']])

bc = PowerTransformer(method='box-cox')
train2['host_acceptance_rateBC']  = yj.fit_transform(train2[['host_acceptance_rate']])
valid2['host_acceptance_rateBC'] = yj.transform(valid2[['host_acceptance_rate']])

In [ ]:
plot(train2, 'host_acceptance_rateYJ')

In [ ]:
# Initialise table
rows = ['No transformation', 'Log transformation', 'Yeo-Johnson transformation','Box-cox transformation']
columns=['RMSE', 'R-Squared', 'MAE']
results = pd.DataFrame(0.0, columns=columns, index=rows)

for i in range(len(rows)):

    # Design matrices
    if i==0:
        predictor = ['host_acceptance_rate']
    elif i==1:
        predictor = ['Loghost_acceptance_rate']
    elif i==2:
        predictor = ['host_acceptance_rateYJ']
    else: 
        predictor = ['host_acceptance_rateBC']
    
    X_train2 =  train2[predictor]
    X_valid2 = valid2[predictor]
    
    # Fit model
    ols = LinearRegression().fit(X_train2, y_train_2)

    # Validation set predictions
    y_pred2 = ols.predict(X_valid2)
    
    # Validation metrics
    results.iloc[i, 0] = np.sqrt(mean_squared_error(y_valid_2, y_pred2))
    results.iloc[i, 1] = r2_score(y_valid_2, y_pred2)
    results.iloc[i, 2] = mean_absolute_error(y_valid_2, y_pred2)

results.round(3)

### 4.2.3 Interaction between response and accept

In [ ]:
train['host_response&accept_rate'] = train['host_response_rate'] * train['host_acceptance_rate']/100
test['host_response&accept_rate'] = test['host_response_rate'] * test['host_acceptance_rate']/100
train['host_response&accept_rate']

In [ ]:
sns.regplot(x = train['host_response&accept_rate'], y = train['LogPrice'], 
            lowess=True, line_kws={'color':'black', 'alpha':0.6})

sns.despine()
plt.show()

In [ ]:
train['Loghost_response&accept_rate'] = np.log(train['host_response&accept_rate'])
test['Loghost_response&accept_rate'] = np.log(test['host_response&accept_rate'])

index_train2, index_valid2 = train_test_split(train.index, train_size=0.7, random_state=21)

train2 = train.loc[index_train2, :].copy()
valid2 = train.loc[index_valid2, :].copy()

train2['Loghost_response&accept_rate'].fillna(0, inplace=True)
valid2['Loghost_response&accept_rate'].fillna(0, inplace=True)
train2['Loghost_response&accept_rate'] = train2['Loghost_response&accept_rate'].replace(-np.inf, 0)
valid2['Loghost_response&accept_rate'] = valid2['Loghost_response&accept_rate'].replace(-np.inf, 0)

y_train_2 = train2['LogPrice']
y_valid_2 = valid2['LogPrice']


sns.regplot(x = train['Loghost_response&accept_rate'], y = train['LogPrice'], 
            lowess=True,  line_kws={'color':'black', 'alpha':0.6})

sns.despine()
plt.show()

In [ ]:
from sklearn.preprocessing import PowerTransformer

yj = PowerTransformer(method='yeo-johnson')
train2['host_response&accept_rateYJ']  = yj.fit_transform(train2[['host_response&accept_rate']])
valid2['host_response&accept_rateYJ'] = yj.transform(valid2[['host_response&accept_rate']])

bc = PowerTransformer(method='box-cox')
train2['host_response&accept_rateBC']  = yj.fit_transform(train2[['host_response&accept_rate']])
valid2['host_response&accept_rateBC'] = yj.transform(valid2[['host_response&accept_rate']])

In [ ]:
# Initialise table
rows = ['No transformation', 'Log transformation', 'Yeo-Johnson transformation','Box-cox transformation']
columns=['RMSE', 'R-Squared', 'MAE']
results = pd.DataFrame(0.0, columns=columns, index=rows)

for i in range(len(rows)):

    # Design matrices
    if i==0:
        predictor = ['host_response&accept_rate']
    elif i==1:
        predictor = ['Loghost_response&accept_rate']
    elif i==2:
        predictor = ['host_response&accept_rateYJ']
    else: 
        predictor = ['host_response&accept_rateBC']
    
    X_train2 =  train2[predictor]
    X_valid2 = valid2[predictor]
    
    # Fit model
    ols = LinearRegression().fit(X_train2, y_train_2)

    # Validation set predictions
    y_pred2 = ols.predict(X_valid2)
    
    # Validation metrics
    results.iloc[i, 0] = np.sqrt(mean_squared_error(y_valid_2, y_pred2))
    results.iloc[i, 1] = r2_score(y_valid_2, y_pred2)
    results.iloc[i, 2] = mean_absolute_error(y_valid_2, y_pred2)

results.round(3)

## 4.3 Longitude and Latitude

In [ ]:
train['Longitude_Latitude'] = (train['longitude'] * train['latitude']).abs()
test['Longitude_Latitude'] = (test['longitude'] * test['latitude']).abs()

In [ ]:
train['Longitude_Latitude'].describe()

In [ ]:
sns.boxplot(y = train['Longitude_Latitude'], boxprops=dict(alpha=.9), palette='Blues')
sns.despine()
plt.show()

In [ ]:
train['LogLongitude_Latitude'] = np.log(train['Longitude_Latitude'])
test['LogLongitude_Latitude'] = np.log(test['Longitude_Latitude'])

index_train3, index_valid3 = train_test_split(train.index, train_size=0.7, random_state=21)

train3 = train.loc[index_train3, :].copy()
valid3 = train.loc[index_valid3, :].copy()

y_train_3 = train3['LogPrice']
y_valid_3 = valid3['LogPrice']

In [ ]:
yj = PowerTransformer(method='yeo-johnson')
train3['Longitude_LatitudeYJ']  = yj.fit_transform(train3[['Longitude_Latitude']])
valid3['Longitude_LatitudeYJ'] = yj.transform(valid3[['Longitude_Latitude']])

bc = PowerTransformer(method='box-cox')
train3['Longitude_LatitudeBC']  = yj.fit_transform(train3[['Longitude_Latitude']])
valid3['Longitude_LatitudeBC'] = yj.transform(valid3[['Longitude_Latitude']])

In [ ]:
# Initialise table
rows = ['No transformation', 'Log transformation', 'Yeo-Johnson transformation','Box-cox transformation']
columns=['RMSE', 'R-Squared', 'MAE']
results = pd.DataFrame(0.0, columns=columns, index=rows)

for i in range(len(rows)):

    # Design matrices
    if i==0:
        predictor = ['Longitude_Latitude']
    elif i==1:
        predictor = ['LogLongitude_Latitude']
    elif i==2:
        predictor = ['Longitude_LatitudeYJ']
    else: 
        predictor = ['Longitude_LatitudeBC']
    
    X_train3 =  train3[predictor]
    X_valid3 = valid3[predictor]
    
    # Fit model
    ols = LinearRegression().fit(X_train3, y_train_3)

    # Validation set predictions
    y_pred3 = ols.predict(X_valid3)
    
    # Validation metrics
    results.iloc[i, 0] = np.sqrt(mean_squared_error(y_valid_3, y_pred3))
    results.iloc[i, 1] = r2_score(y_valid_3, y_pred3)
    results.iloc[i, 2] = mean_absolute_error(y_valid_3, y_pred3)

results.round(3)

# 5. Date Variable

In [ ]:
data['host_since'].dtype

In [ ]:
data['host_since'] = pd.to_datetime(data['host_since'])

In [ ]:
data['host_since'].isna().sum()

In [ ]:
tr_day_to_now = pd.to_datetime('2023-12-31') - pd.to_datetime(train['host_since'])
train['days_to_now'] = tr_day_to_now.dt.days

In [ ]:
te_day_to_now = pd.to_datetime('2023-12-31') - pd.to_datetime(test['host_since'])
test['days_to_now'] = te_day_to_now.dt.days

In [ ]:
train['days_to_now'].head()

In [ ]:
train['first_review'].isna().sum()

In [ ]:
train['first_review'].head()

In [ ]:
train['last_review'].isna().sum()

In [ ]:
# drop the 'days'
tr_day_reviews = pd.to_datetime(train['last_review']) - pd.to_datetime(train['first_review'])
tr_day_reviews = tr_day_reviews.apply(lambda x: str(x.days) if not pd.isna(x) else np.nan)

# replace nan to -1 as 'Notreviewed'
train['day_reviews'] = tr_day_reviews.fillna(-1)

train['day_reviews'] = train['day_reviews'].astype(float)
train['day_reviews']

In [ ]:
# drop the 'days'
te_day_reviews = pd.to_datetime(test['last_review']) - pd.to_datetime(test['first_review'])
te_day_reviews = te_day_reviews.apply(lambda x: str(x.days) if not pd.isna(x) else np.nan)

# replace nan to -1 as'NotReviewed'
test['day_reviews'] = te_day_reviews.fillna(-1)
test['day_reviews'] = test['day_reviews'].astype(float)

In [ ]:
train['day_reviews'].isna().sum()

In [ ]:
train['day_reviews'].head()

# 6. Correlation

In [ ]:
New_variable=['review_scores_rating']

In [ ]:
####
plt.figure(figsize=(12, 10))  
Selected_discrete_variable=['host_total_listings_count','accommodates','minimum_nights','minimum_maximum_nights' ,'availability_60','number_of_reviews','calculated_host_listings_count']
corr = train[Selected_discrete_variable + Continuous_Variable + New_variable + Response].corr()


plt.imshow(corr, cmap='coolwarm', vmin=-1, vmax=1) 


plt.xticks(range(len(corr)), corr.columns, rotation=90)  
plt.yticks(range(len(corr)), corr.columns)


plt.colorbar()

plt.title('Correlations')
plt.savefig('correlation.png')
plt.show()

In [ ]:
select_variable = train[Selected_discrete_variable + Continuous_Variable + New_variable + Response]

In [ ]:
sns.pairplot(select_variable)

### The latitude and longtude may affect different areas on the selected variables

# 7. Text Variable

In [ ]:
data[Text_Variable]

In [ ]:
train[Text_Variable].isna().sum()

In [ ]:
train['neighborhood_overview'] =train['neighborhood_overview'].fillna('NAN')
test['neighborhood_overview'] =test['neighborhood_overview'].fillna('NAN')

train['host_about'] =train['host_about'].fillna('NAN')
test['host_about'] =test['host_about'].fillna('NAN')

In [ ]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [ ]:
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

In [ ]:
import re 

def tokenizer(text):
    
    # Replace emojis with special tokens
    text = re.sub(r'[❤💕💗💜💖💘💙💝😍]', ' xxheart ', text)
    text = re.sub(r'[😀😉😄😎😊😁🌞☺]', ' xxsmile ', text) 
    text = re.sub(r'[😢😔😭😞😪😕😓😩😦]', ' xxsadface ', text) 
    text = re.sub(r'[😋😜]', ' xxlolface ', text) 
    
    # Replace urls with a special token
    text = re.sub('https?\S+', 'xxurl', text) 
    text = re.sub('\S*www[.]\S+', 'xxurl', text)

    # Replace emoticons with special tokens
    text = re.sub(r'<3', 'xxheart', text) 
    text = re.sub(r'&lt;3', 'xxheart', text)
    
    eyes = r'[8:=;]'
    nose = r'[\'`-]*'
    text = re.sub(eyes + nose + r'[)D]+', 'xxsmile', text)
    text = re.sub(eyes + nose + r'[(]+', 'xxsadface', text)
    text = re.sub(eyes + nose + r'p+', 'xxlolface', text)
    text = re.sub(eyes + nose + r'[|/\\]+', 'xxneutralface', text)

    # Creating a special token for hashtags
    text = re.sub(r'#(\S+)', 'xxhashtag ' + r'\1', text) # hashtag
    
    # Stripping repeated whitespace
    text = re.sub(r'\s{2,}', ' ', text)
    
    doc = nlp(text)
    
    tokens = []
    
    for token in doc: 
        
        word = token.lemma_.lower()
        
        if not token.is_stop:
            
            if word == '!': 
                tokens.append('!') 
                            
                
            elif (not token.is_punct) and word != '️':
                tokens.append(word)
                
    return tokens

In [ ]:
tokens_neigh = train['neighborhood_overview'].apply(tokenizer)

print(tokens_neigh)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 

def identity_tokenizer(text):
    return text

tfidf = TfidfVectorizer(tokenizer=identity_tokenizer,
                        max_features =15,max_df =0.9,
                        stop_words='english',lowercase=False,ngram_range = (2,2))

corpus_neigh = train['neighborhood_overview'].apply(tokenizer) 
test_cor_neigh = test['neighborhood_overview'].apply(tokenizer) 

tfidf.fit(corpus_neigh)

train_text_neigh = tfidf.transform(corpus_neigh).todense() 
test_text_neigh = tfidf.transform(test_cor_neigh).todense() 
name_list_neigh = tfidf.get_feature_names_out()

train_text_neigh = pd.DataFrame(train_text_neigh,columns = name_list_neigh) 
test_text_neigh = pd.DataFrame(test_text_neigh,columns = name_list_neigh)

In [ ]:
train_text_neigh.info()


# 8. Response Variable

In [ ]:
train['price'].describe()

In [ ]:
test['price'].describe()

In [ ]:
outlier_price = test[test['price']==100000]

In [ ]:
outlier_price[['price','accommodates','bathrooms_text','neighbourhood_cleansed']]

In [ ]:
index_of_max = test['LogPrice'].idxmax()
index_of_max

In [ ]:
test = test.drop(index_of_max)

In [ ]:
test['price'].describe()

# 9.0 Train and Test

In [ ]:
y_train = train['LogPrice']

encoder = CatBoostEncoder().fit(train['neighbourhood_cleansed'], y_train) 
train['neighbourhood_cleansed_cat'] = encoder.transform(train['neighbourhood_cleansed'])

In [ ]:
y_test = test['LogPrice']

encoder = CatBoostEncoder().fit(test['neighbourhood_cleansed'], y_test) 
test['neighbourhood_cleansed_cat'] = encoder.transform(test['neighbourhood_cleansed']) 

In [ ]:
yj = PowerTransformer(method='yeo-johnson')
train['host_response_rate_yj']  = yj.fit_transform(train[['host_response_rate']])
test['host_response_rate_yj'] = yj.transform(test[['host_response_rate']])


train['host_acceptance_rate_yj']  = yj.fit_transform(train[['host_acceptance_rate']])
test['host_acceptance_rate_yj'] = yj.transform(test[['host_acceptance_rate']])


train['host_response&accept_rate_yj']  = yj.fit_transform(train[['host_response&accept_rate']])
test['host_response&accept_rate_yj'] = yj.transform(test[['host_response&accept_rate']])

In [ ]:
train.info()

In [ ]:
train.drop(columns=['neighborhood_overview','source','host_since','host_location','host_is_superhost','host_neighbourhood','neighbourhood_cleansed','property_type','room_type',
                    'Loghost_response&accept_rate', 'LogLongitude_Latitude','neighbourhood',
                    'first_review','last_review','host_response_rate','host_acceptance_rate',
                    'Loghost_response_rate','Loghost_acceptance_rate','latitude','longitude',
                    'host_response_rate_yj','host_acceptance_rate_yj','host_response&accept_rate',
                    'name','host_about'], inplace=True)

In [ ]:
test.drop(columns=['neighborhood_overview','source','host_since','host_location','host_is_superhost','host_neighbourhood','neighbourhood_cleansed','property_type','room_type',
                    'Loghost_response&accept_rate', 'LogLongitude_Latitude','neighbourhood',
                    'first_review','last_review','host_response_rate','host_acceptance_rate',
                    'Loghost_response_rate','Loghost_acceptance_rate','latitude','longitude',
                    'host_response_rate_yj','host_acceptance_rate_yj','host_response&accept_rate',
                    'name','host_about'], inplace=True)

In [ ]:
train.reset_index(drop=True, inplace=True)
train_text_neigh.reset_index(drop=True, inplace=True)
train = pd.concat([train, train_text_neigh], axis=1)

In [ ]:
test.reset_index(drop=True, inplace=True)
test_text_neigh.reset_index(drop=True, inplace=True)
test = pd.concat([test, test_text_neigh], axis=1)

In [ ]:
index_train, index_valid = train_test_split(train.index, train_size=0.7, random_state=21)

sub_train = train.loc[index_train, :].copy()
sub_valid = train.loc[index_valid, :].copy()

y_sub_train = sub_train['LogPrice']
y_sub_valid = sub_valid['LogPrice']

In [ ]:
x_sub_train = sub_train.drop(columns=['LogPrice','price'])
x_sub_valid = sub_valid.drop(columns=['LogPrice','price'])

In [ ]:
x_sub_train.drop(columns=['/><br />the','< br','br />-','br /><br'], inplace=True)

In [ ]:
x_sub_valid.drop(columns=['/><br />the','< br','br />-','br /><br'], inplace=True)

In [ ]:
train.drop(columns=['/><br />the','< br','br />-','br /><br'], inplace=True)

In [ ]:
test.drop(columns=['/><br />the','< br','br />-','br /><br'], inplace=True)

In [ ]:
test.info()

In [ ]:
test = test.iloc[:-1]

In [ ]:
train.info()

In [ ]:
x_sub_train["Longitude_Latitude"].isna().sum()

# 10.0 Linear model

## 10.1 Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
ols=LinearRegression()
ols.fit(x_sub_train,y_sub_train)

## 10.2 Linear regression with ridge regression


In [ ]:
from sklearn.linear_model import RidgeCV
%time
alphas=list(np.logspace(-15,15,151,base=2))
ridge=RidgeCV(alphas=alphas, cv = 5)
ridge.fit(x_sub_train,y_sub_train)

In [ ]:
ridge.alpha_

In [ ]:
from tutorial4 import coefplot
coefplot(ridge, x_sub_train.columns)
plt.show()

## 10.3 Linear regression with lasso


In [ ]:
from sklearn.linear_model import LassoCV
%time
alphas=list(np.logspace(-15,15,151,base=2))
lasso=LassoCV(alphas = alphas, cv=5)
lasso.fit(x_sub_train,y_sub_train)

In [ ]:
lasso.alpha_

In [ ]:
from tutorial4 import coefplot
coefplot(lasso, x_sub_train.columns)
plt.show()

## 10.4 Linear regression with Elastic net


In [ ]:
from sklearn.linear_model import ElasticNetCV
elastic=ElasticNetCV(l1_ratio=[0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99], cv=5)
elastic.fit(x_sub_train,y_sub_train)
print(elastic.l1_ratio_)

In [ ]:
from tutorial4 import coefplot
coefplot(elastic, x_sub_train.columns)
plt.show()

In [ ]:
columns=['RMSE', 'R-Squared', 'MAE']
rows=['Linear Regression', 'OLS $\ell_1$','OLS $\ell_2$','OLS Elastic net']

results =pd.DataFrame(0.0, columns=columns, index=rows)

methods = [ols, lasso, ridge, elastic]

y_pred = np.zeros((len(y_sub_valid), 4))
residual = np.zeros((len(y_sub_train), 4))
avgexp_res = np.zeros((1, 4))

for i, method in enumerate(methods):
    
    residual[:,i] = y_sub_train - method.predict(x_sub_train)
    avgexp_res[:,i] = np.mean(np.exp(residual[:,i]))
    y_pred[:, i] = np.exp(method.predict(x_sub_valid)) * avgexp_res[:,i]
    
for i in range(len(rows)):
    results.iloc[i, 0] = np.sqrt(mean_squared_error(np.exp(y_sub_valid), y_pred[:, i]))
    results.iloc[i, 1] = r2_score(np.exp(y_sub_valid), y_pred[:, i])
    results.iloc[i, 2] = mean_absolute_error(np.exp(y_sub_valid), y_pred[:, i])

results.round(3)

# 11.0 KNN

In [ ]:
from sklearn.model_selection import RepeatedKFold

rkf = RepeatedKFold(n_splits=5,n_repeats=10,random_state=50)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score

model = KNeighborsRegressor(metric='mahalanobis', metric_params={'V': x_sub_train.cov()}) 

param_grid = {'n_neighbors': np.arange(1,51),}

knn =  GridSearchCV(model, param_grid, cv = rkf,  scoring = 'neg_mean_squared_error', verbose=1)
knn.fit(x_sub_train, y_sub_train)

In [ ]:
knn.best_params_

In [ ]:
knn.best_estimator_

 # 12.0 Generalised Additive Model

In [ ]:
from sklearn.preprocessing import SplineTransformer
from pygam import LinearGAM, s, f, l, te
from optuna.samplers import TPESampler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
y_sub_train = np.exp(y_sub_train)
y_sub_valid = np.exp(y_sub_valid)

In [ ]:
gam = LinearGAM(s(0) + s(1) + s(2) + l(3) + s(4) + s(5) + s(6) + s(7) + s(8) + s(9) + s(10) + 
                      s(11) + s(12) + s(13) + s(14) + l(15) + l(16) + l(17) + l(18) + l(19) + l(20) + 
                      l(21) + l(22) + l(23) + l(24) + l(25) + l(26) + l(27) + l(28) + l(29) + l(30) + 
                      s(31) + l(32) + l(33) + l(34) + l(35) + l(36) +l(37)+ l(38) + l(39) + l(40) + 
                      l(41) + l(42) + l(43) + l(44) + l(45) + l(46) + l(47) + l(48) + l(49) + 
                      l(50) + l(51) + l(52) + l(53) + l(54) + l(55) + l(56) + l(57) + l(58) + 
                      l(59) + l(60) + l(61) + l(62) + l(63) + l(64) + l(65) + l(66) + l(67) + 
                      l(68) + l(69) + l(70) + l(71) + l(72) + l(73) + l(74) + s(75) + s(76) + 
                      s(77) + s(78) + s(79) + l(80) + l(81) + l(82) + l(83) + l(84) + l(85) + 
                      l(86) + l(87) + l(88))
                
gam.fit(x_sub_train, y_sub_train)



def objective(trial):
    lambdas = [trial.suggest_loguniform(f'lambda_{j}', 1e-4, 10000.0) for j in range(89)]
    
    
    
    model = LinearGAM(s(0) + s(1) + s(2) + l(3) + s(4) + s(5) + s(6) + s(7) + s(8) + s(9) + s(10) + 
                      s(11) + s(12) + s(13) + s(14) + l(15) + l(16) + l(17) + l(18) + l(19) + l(20) + 
                      l(21) + l(22) + l(23) + l(24) + l(25) + l(26) + l(27) + l(28) + l(29) + l(30) + 
                      s(31) + l(32) + l(33) + l(34) + l(35) + l(36) +l(37)+ l(38) + l(39) + l(40) + 
                      l(41) + l(42) + l(43) + l(44) + l(45) + l(46) + l(47) + l(48) + l(49) + 
                      l(50) + l(51) + l(52) + l(53) + l(54) + l(55) + l(56) + l(57) + l(58) + 
                      l(59) + l(60) + l(61) + l(62) + l(63) + l(64) + l(65) + l(66) + l(67) + 
                      l(68) + l(69) + l(70) + l(71) + l(72) + l(73) + l(74) + s(75) + s(76) + 
                      s(77) + s(78) + s(79) + l(80) + l(81) + l(82) + l(83) + l(84) + l(85) + 
                      l(86) + l(87) + l(88),lam=lambdas)
    
    
    
    
    model.fit(x_sub_train, y_sub_train)
    return model.statistics_['GCV']

sampler = TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=10000, timeout=3000)

lambdas = list(study.best_params.values())



gam_optimized = LinearGAM(s(0) + s(1) + s(2) + l(3) + s(4) + s(5) + s(6) + s(7) + s(8) + s(9) + s(10) + 
                      s(11) + s(12) + s(13) + s(14) + l(15) + l(16) + l(17) + l(18) + l(19) + l(20) + 
                      l(21) + l(22) + l(23) + l(24) + l(25) + l(26) + l(27) + l(28) + l(29) + l(30) + 
                      s(31) + l(32) + l(33) + l(34) + l(35) + l(36) +l(37)+ l(38) + l(39) + l(40) + 
                      l(41) + l(42) + l(43) + l(44) + l(45) + l(46) + l(47) + l(48) + l(49) + 
                      l(50) + l(51) + l(52) + l(53) + l(54) + l(55) + l(56) + l(57) + l(58) + 
                      l(59) + l(60) + l(61) + l(62) + l(63) + l(64) + l(65) + l(66) + l(67) + 
                      l(68) + l(69) + l(70) + l(71) + l(72) + l(73) + l(74) + s(75) + s(76) + 
                      s(77) + s(78) + s(79) + l(80) + l(81) + l(82) + l(83) + l(84) + l(85) + 
                      l(86) + l(87) + l(88),lam=lambdas)


gam_optimized.fit(x_sub_train, y_sub_train)



def evaluate_model(model, X, y_sub_valid):
    y_pred = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y_sub_valid, y_pred))
    r2 = r2_score(y_sub_valid, y_pred)
    mae = mean_absolute_error(y_sub_valid, y_pred)
    return rmse, r2, mae

rmse_gam, r2_gam, mae_gam = evaluate_model(gam_optimized, x_sub_valid, y_sub_valid)

results = pd.DataFrame({
    'Model': ['GAM'],
    'RMSE': [rmse_gam,],
    'R-Squared': [r2_gam],
    'MAE': [mae_gam]
})



In [ ]:
results

# 13.0 Tree model

# 13.1 Decision tree

## 13.1.1 Growing the tree

In [ ]:
feature_names = x_sub_train.columns.tolist()

In [ ]:
from sklearn.tree import DecisionTreeRegressor

reg = DecisionTreeRegressor(criterion='squared_error', max_leaf_nodes=10)

reg.fit(x_sub_train, y_sub_train)

In [ ]:
import graphviz
from sklearn.tree import export_graphviz

dot_data = export_graphviz(reg, out_file=None , impurity=False, feature_names = feature_names, 
                           rounded=True) 
graph = graphviz.Source(dot_data)
graph

In [ ]:
from sklearn.tree import plot_tree

plot_tree(reg, feature_names = feature_names,  impurity=False,
          rounded=True, filled = True)
plt.show()

In [ ]:
import dtreeviz
viz = dtreeviz.model(reg,  x_sub_train, y_sub_train,target_name = 'LogPrice', feature_names = feature_names)  
               
viz.view(scale=2)

In [ ]:
reg = DecisionTreeRegressor(criterion='squared_error', max_leaf_nodes=10)
reg.fit(x_sub_train, y_sub_train)

dot_data = export_graphviz(reg, out_file=None , impurity=False, feature_names = feature_names, rounded=True) 
graph = graphviz.Source(dot_data)
graph

In [ ]:
viz = dtreeviz.model(reg,  x_sub_train, y_sub_train, feature_names = feature_names)  
              
viz.view(scale=2)

## 13.1.2 Cost-complexity pruning

In [ ]:
model = DecisionTreeRegressor(criterion='squared_error', min_samples_leaf=10)

path = model.cost_complexity_pruning_path(x_sub_train, y_sub_train)

alphas = path.ccp_alphas

In [ ]:
impurities = path.impurities
impurities[:5]

In [ ]:
print(len(alphas))
print(alphas[:10])

plt.plot(alphas)
#plt.ylabel('alpha')


plt.plot(impurities)
plt.legend(['ccp_alphas','impurities'] )
plt.show()

In [ ]:
ccp_alphas, impurities = path.ccp_alphas, path.impurities

'''
ccp_alphas: also called effective alphas, measures the strenght of link of a node to the tree.  
Nodes with the smallest effective alpha have the “weakest link” and are pruned first.
'''
fig, ax = plt.subplots()
ax.plot(ccp_alphas[:-1], impurities[:-1], marker="o", drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set")

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
model = DecisionTreeRegressor(criterion='squared_error', min_samples_leaf=5)

search_space = {
    'ccp_alpha': alphas,
}

tree_search = RandomizedSearchCV(model, search_space, cv = 5)
tree_search.fit(x_sub_train, y_sub_train)
tree = tree_search.best_estimator_

print('Best parameters found by grid search:', tree_search.best_params_, '\n')

In [ ]:
dot_data = export_graphviz(reg, out_file=None , impurity=False, feature_names = feature_names,
                           rounded=True) 
graph = graphviz.Source(dot_data)
graph.render('Optimal Tree') # saves tree to a file
graph

In [ ]:
n_nodes=tree.tree_.node_count
print('Number of leaves: ', n_nodes)

## 13.2 Bagging

In [ ]:
from sklearn.ensemble import BaggingRegressor

bag = BaggingRegressor(DecisionTreeRegressor(criterion='squared_error'), n_estimators=1000, random_state=42)
bag.fit(x_sub_train, y_sub_train)

## 13.3 Random forest

### 13.3.1 Fitting a random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf =  RandomForestRegressor(n_estimators=100,  criterion='squared_error',  max_features = 20, min_samples_leaf= 1)

rf.fit(x_sub_train, y_sub_train)

### 13.3.2 Hyperparameter optimisation

In [ ]:
import optuna
from optuna.samplers import TPESampler

#suppress log messages
optuna.logging.set_verbosity(optuna.logging.WARNING)

from sklearn.model_selection import cross_val_score

def objective(trial):
    
    criterion = trial.suggest_categorical('criterion', ['squared_error', 'absolute_error'])
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    max_features = trial.suggest_int('max_features', 1, 20)
    
    
    model = RandomForestRegressor(n_estimators = 100,  
                                  criterion = criterion,
                                  max_features = max_features, 
                                  min_samples_leaf= min_samples_leaf,
                                  random_state = 1)
    
    scores = cross_val_score(model, x_sub_train, y_sub_train, cv = 5, scoring = 'r2')
    loss = 1 - np.mean(scores)
    
    return loss

sampler = TPESampler(seed=42) 
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials = 100, timeout = 3000) 

# the timeout should be as high as possible in a real application

In [ ]:
#convert output to a dataframe.
study_df=study.trials_dataframe()
trials_df=study_df.drop(columns=['datetime_start', 'datetime_complete','duration' ])
print('Size of output:', trials_df.shape)
display('Trial info: ', trials_df.head())
display('Best trial info: ', trials_df[trials_df.value == trials_df.value.min()])

In [ ]:
params = study.best_params
print(type(params)) #dictionary
print(params)

In [ ]:
 # **params "unpacks" the dictionary and passes the items as inputs to the random forest class
    
rf = RandomForestRegressor(n_estimators=1000, **params) 
rf.fit(x_sub_train, y_sub_train)

In [ ]:
from tutorial9 import plot_feature_importance
plot_feature_importance(rf,feature_names)
plt.show()

## 13.5 Validation results

In [ ]:
columns=['Test RMSE', 'Test R$^2$', 'Test MAE']
rows=['Regressor Tree', 'Bagged trees', 'Random forest']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

methods=[tree, bag, rf]

for i, method in enumerate(methods):
    
     y_pred = method.predict(x_sub_valid)
    
     results.iloc[i,0] = np.sqrt(mean_squared_error(y_sub_valid, y_pred))
     results.iloc[i,1] = r2_score(y_sub_valid, y_pred)
     results.iloc[i,2] = mean_absolute_error(y_sub_valid, y_pred)
   

results.round(3)

# 14.0 Boosting

## 14.1Scikit-Learn

In [ ]:
%%time

from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor(learning_rate = 0.01, max_depth = 8, n_estimators= 1901, subsample = 0.596)
gb.fit(x_sub_train, y_sub_train)

In [ ]:
plot_feature_importance(gb, feature_names)
plt.show()

In [ ]:
%%time

from sklearn.ensemble import HistGradientBoostingRegressor

hgb = HistGradientBoostingRegressor(learning_rate = 0.01, max_depth = 8, max_iter = 1901)
hgb.fit(x_sub_train, y_sub_train)

## 14.2 XGBoost

In [ ]:
%%time

import xgboost as xgb

xgbst = xgb.XGBRegressor(objective='reg:squarederror', booster = 'gbtree', learning_rate = 0.02, 
                        max_depth = 6, n_estimators = 1822, subsample = 0.60,  reg_lambda=0.0, 
                        reg_alpha = 0.0, colsample_bytree = 1.0, min_child_weight = 1.0)

# n_estimators: number of trees
# subsample: fraction of observations used to fit each tree
# reg_lambda: L2 regularisation penalty
# reg_alpha: L1 regularisation penalty
# colsample_bytree: fraction of features considered for each tree (alternatives: bylevel, bynode)
# min_child_weight: approximately equal to the minimum number of samples in each node

xgbst.fit(x_sub_train, y_sub_train)

In [ ]:
plot_feature_importance(xgbst, labels=feature_names)
plt.show()

## 14.3 LightGBM

In [ ]:
%%time

import lightgbm as lgb

lgbm = lgb.LGBMRegressor(objective='regression', boosting_type = 'gbdt', 
                         n_estimators = 1974, learning_rate = 0.01,  
                         num_leaves = 24,
                         lambda_l1 = 0.0, lambda_l2 = 0.0,
                         bagging_fraction = 0.74, bagging_freq = 1, 
                         feature_fraction = 1.0, min_data_in_leaf = 1)

lgbm = lgbm.fit(x_sub_train, y_sub_train)

In [ ]:
plot_feature_importance(lgbm, labels=feature_names)
plt.show()

### Further LightGBM

### 14.3.1 Training API

In [ ]:
train_data = lgb.Dataset(x_sub_train, y_sub_train)
valid_data = lgb.Dataset(x_sub_valid, y_sub_valid, reference = train_data)

# An important detail that is not applicable to this tutorial is that LightGBM can handle
# categorical features in a specialised way without the need for prior encoding.

# Suppose that that the first two columns of X_train are categorical. You'd then run:
# train_data = lgb.Dataset(X_train, y_train, categorical_feature = [0, 1])

# Alternatively, you can pass the list of categorical features to the train function below

In [ ]:
%%time

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'metric': 'l2',
    'learning_rate': 0.01,
    'num_leaves': 31,
    'lambda_l1': 0.0, 
    'lambda_l2': 0.0, 
    'bagging_fraction': 0.9,
    'bagging_freq': 5,
    'feature_fraction': 0.9,
    'min_data_in_leaf': 20, 
}

evals = {} 
lgbm = lgb.train(params, train_data, num_boost_round = 2000, valid_sets = [train_data, valid_data], 
                 valid_names=['Training', 'Validation'], callbacks = {lgb.record_evaluation(evals)})

In [ ]:
lgb.plot_metric(evals, metric = 'l2')
sns.despine()
plt.show()

### 14.3.2 Early stopping

In [ ]:
%%time

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'learning_rate': 0.01,
    'num_leaves': 31,
    'lambda_l1': 0.0, 
    'lambda_l2': 0.0, 
    'bagging_fraction': 0.9,
    'bagging_freq': 5,
    'feature_fraction': 0.9,
    'min_data_in_leaf': 20
}

evals = {} 

lgbm = lgb.train(params, train_data, num_boost_round = 5000,  valid_sets = [train_data, valid_data], 
                 valid_names=['Training', 'Validation'], callbacks = [lgb.early_stopping(stopping_rounds=100), lgb.record_evaluation(evals)])

### 14.3.3 Cross validation

In [ ]:
%%time

params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'num_iterations': 5000,
    'learning_rate': 0.01,
    'num_leaves': 31,
    'lambda_l1': 0.0, 
    'lambda_l2': 0.0, 
    'bagging_fraction': 0.9,
    'bagging_freq': 5,
    'feature_fraction': 0.9,
    'min_data_in_leaf': 20, 
    'verbosity': -1
}


history = lgb.cv(params, train_data, num_boost_round = 5000, 
                 nfold = 5, stratified = False, callbacks = [lgb.early_stopping(stopping_rounds=100)])

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

plt.plot(history['valid l2-mean'])

ax.set_ylabel('Cross-validation MSE')
ax.set_xlabel('Boosting iterations')

sns.despine()
plt.show()

### 14.3.4 Hyperparameter optimisation

In [ ]:
# We have to reconstruct the dataset to tune min_data_in_leaf
train_data = lgb.Dataset(x_sub_train, y_sub_train)
valid_data = lgb.Dataset(x_sub_valid, y_sub_valid, reference = train_data)

from optuna.samplers import TPESampler

def objective(trial):
    
    params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'learning_rate': 0.01, 
    'num_leaves': trial.suggest_int('num_leaves', 2, 64),
    'lambda_l1': trial.suggest_loguniform('lambda_l1',  1e-8, 10), 
    'lambda_l2': trial.suggest_loguniform('lambda_l2',  1e-8, 10), 
    'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.5, 1.0),
    'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
    'feature_fraction':  trial.suggest_uniform('feature_fraction', 0.5, 1.0),
    'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 128), 
    'feature_pre_filter': False,
    'verbosity': -1,
     }
    
    # Cross-validation 
    history = lgb.cv(params, train_data, num_boost_round = 5000, 
                 nfold = 5, stratified = False, callbacks = [lgb.early_stopping(stopping_rounds = 50)])
    
    # Save full set of parameters
    trial.set_user_attr('params', params)
    
    # Save the number of boosting iterations selected by early stopping
    trial.set_user_attr('num_boost_round', len(history['valid l2-mean']))
    
    return history['valid l2-mean'][-1] # returns CV error for the best trial

sampler = TPESampler(seed = 42) 
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=500, timeout = 3000)  

# Important! The higher the timeout, the better the chances of finding a good configuration. 
# The time budget should be much higher in a real application.

In [ ]:
params = study.best_trial.user_attrs['params']
num_trees = study.best_trial.user_attrs['num_boost_round']

print(f'Number of boosting iterations: {num_trees} \n')

print('Best parameters:')
params 

In [ ]:
importance = optuna.importance.get_param_importances(study)

importance

### 14.3.5 Optuna integration

In [ ]:
# skip running this cell if you want to save time

from optuna.integration import LightGBMPruningCallback

def objective(trial):
    
    params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': trial.suggest_int('num_leaves', 2, 64),
    'lambda_l1': trial.suggest_loguniform('lambda_l1',  1e-8, 10.0), 
    'lambda_l2': trial.suggest_loguniform('lambda_l2',  1e-8, 10.0), 
    'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.5, 1.0),
    'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
    'feature_fraction':  trial.suggest_uniform('feature_fraction', 0.3, 1.0),
    'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 128),
    'feature_pre_filter': False,
    'verbosity': -1,
     }
    
    # Instantiate pruning callback
    pruning_callback =  LightGBMPruningCallback(trial, 'l2')
    
    # LightGBM cross-validation
    history = lgb.cv(params, train_data, num_boost_round = 5000, 
                 nfold = 5, stratified = False, 
                 callbacks = [lgb.early_stopping(stopping_rounds = 50), pruning_callback])
    
    # Save full set of parameters
    trial.set_user_attr('params', params)
    
    # Save the number of boosting iterations selected by early stopping
    trial.set_user_attr('num_boost_round', len(history['valid l2-mean']))
    
    return history['valid l2-mean'][-1] # returns CV error for the best trial

# Instantiating the sampler to set the random seed
sampler = TPESampler(seed=42) 

# Instantiating the pruner
# The min_resource should be the minimum number of trees required to reasonably judge performance
# This can be tricky to get right for boosting
# See the notes in the documentation:
# https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.HyperbandPruner.html
pruner = optuna.pruners.HyperbandPruner(min_resource = 100, max_resource = 'auto', reduction_factor = 3)

# A simpler alternative is the median pruner
# This method prunes trials with partial performance below the median of previous trials 
#pruner = optuna.pruners.MedianPruner(n_startup_trials = 10, n_warmup_steps = 500, interval_steps = 5)

study = optuna.create_study(direction='minimize', sampler = sampler, pruner = pruner)
study.optimize(objective, n_trials = 500, timeout= 3000)

In [ ]:
params = study.best_trial.user_attrs['params']
num_trees = study.best_trial.user_attrs['num_boost_round']

print(f'Number of boosting iterations: {num_trees} \n')

print('Best parameters:')
params 

In [ ]:
# skip running this cell if you want to save time

import optuna.integration.lightgbm as lgb

dtrain = lgb.Dataset(x_sub_train, y_sub_train)
    
params = {
    'objective': 'regression',
    'metric': 'l2',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'feature_pre_filter': False,
    'verbosity': -1,
}

tuner = lgb.LightGBMTunerCV(params, dtrain, num_boost_round = 5000, callbacks = [lgb.early_stopping(stopping_rounds = 50)], 
                            time_budget = 3000, nfold=5, stratified = False)

tuner.run()

In [ ]:
print('Best params:')

tuner.best_params

### 14.3.6 Final LightGBM model

In [ ]:
import lightgbm as lgb

params = {
 'objective': 'regression',
 'boosting_type': 'gbdt',
 'learning_rate': 0.01,
 'num_leaves': 45,
 'lambda_l1': 0.027109795180157614,
 'lambda_l2': 1.3211122996540583e-06,
 'bagging_fraction': 0.9303390497096511,
 'bagging_freq': 8,
 'feature_fraction': 0.6954068017048125,
 'min_data_in_leaf': 2}

lgbm = lgb.train(params, train_data, num_boost_round = 3996)

In [ ]:
lgbm = lgb.LGBMRegressor(objective= 'regression',
 metric= 'l2',
 boosting_type= 'gbdt',
 learning_rate= 0.05,
 feature_pre_filter= False,
 verbosity= -1,
 lambda_l1= 2.580935030198812e-06,
 lambda_l2= 3.4697812081022227e-06,
 num_leaves= 84,
 feature_fraction= 0.4,
 bagging_fraction= 1.0,
 bagging_freq= 0,
 min_child_samples= 20)

lgbm = lgbm.fit(x_sub_train, y_sub_train)

In [ ]:
plot_feature_importance(lgbm, labels=feature_names)
plt.show()

## 14.4 CatBoost

In [ ]:
%%time

from catboost import CatBoostRegressor

cb = CatBoostRegressor(loss_function='RMSE', 
                       iterations = 2000, learning_rate = 0.01,
                       l2_leaf_reg = 0.0,
                       max_depth = 6, 
                       subsample = 0.8, 
                       verbose=0)
        
cb.fit(x_sub_train, y_sub_train)

# Important! If there were categorical features, you'd want to change the code to
# cb.fit(X_train, y_train, cat_features = cat_indices)
# where cat_indices is a one-dimensional array of indices indicating the categorical columns in X_train

In [ ]:
cb = CatBoostRegressor(loss_function='RMSE', verbose=0)        
cb.fit(x_sub_train, y_sub_train)

## 14.5 Model selection

In [ ]:
# Auxiliary code - important to understand but similar to previous tutorials

columns=['RMSE', 'R$^2$', 'MAE']
rows=[ 'GBM', 'Histogram-based GBM',
      'XGBoost', 'CatBoost', 'LightGBM']

results=pd.DataFrame(0.0, columns=columns, index=rows) 

methods = [gb, hgb, xgbst, cb, lgbm]

for i, method in enumerate(methods):

    y_pred = method.predict(x_sub_valid)
    
    results.iloc[i,0] = np.sqrt(mean_squared_error(y_sub_valid, y_pred))
    results.iloc[i,1] = r2_score(y_sub_valid, y_pred)
    results.iloc[i,2] = mean_absolute_error(y_sub_valid, y_pred)

results.round(3)

# 15.0 Model stack

In [ ]:
from sklearn.ensemble import StackingRegressor
 
stack = StackingRegressor(estimators = [('LightGBM', lgbm),
                                        ('Random forest', rf),('Catboost', cb)], final_estimator = LinearRegression(), 
                           cv = 5)

stack.fit(x_sub_train, y_sub_train)

In [ ]:
stack.final_estimator_.coef_

In [ ]:
# Initialise table
columns=['RMSE', 'R-Squared', 'MAE']
rows=[ 'LightGBM', 'Random forest','Catboost' ,'Stack']

results =pd.DataFrame(0.0, columns=columns, index=rows)

# Methods
methods = [lgbm,rf,cb,stack]

# Compute test predictions
y_pred = np.zeros((len(y_sub_valid), 4))

for i, method in enumerate(methods):
    
    y_pred[:, i] = method.predict(x_sub_valid)
    
# Validation results
for i in range(len(rows)):
    results.iloc[i, 0] = np.sqrt(mean_squared_error(y_sub_valid, y_pred[:, i]))
    results.iloc[i, 1] = r2_score(y_sub_valid, y_pred[:, i])
    results.iloc[i, 2] = mean_absolute_error(y_sub_valid, y_pred[:, i])

results.round(3)

# 16.0 Model assessment

In [ ]:
test.info()

In [ ]:
y_test = test['LogPrice']

In [ ]:
x_test = test.drop(columns=['LogPrice','price'])

In [ ]:
y_test = np.exp(y_test)

In [ ]:
X_Train = pd.concat([x_sub_train, x_sub_valid], axis=0)

In [ ]:
Y_Train = pd.concat([y_sub_train, y_sub_valid], axis=0)

In [ ]:
stack.fit(X_Train, Y_Train)

In [ ]:
columns=['RMSE', 'R-Squared', 'MAE']
rows=['Stack']

results =pd.DataFrame(0.0, columns=columns, index=rows)

# Methods
methods = [stack]

# Compute test predictions
y_pred = np.zeros((len(y_test), 1))
                  
for i, method in enumerate(methods):
    
    y_pred[:, i] = method.predict(x_test)
    
# Validation results
for i in range(len(rows)):
    results.iloc[i, 0] = np.sqrt(mean_squared_error(y_test, y_pred[:, i]))
    results.iloc[i, 1] = r2_score(y_test, y_pred[:, i])
    results.iloc[i, 2] = mean_absolute_error(y_test, y_pred[:, i])

results.round(3)